# 根据用户特征和电影特征onehot/multihot编码后进行推荐

# 尝试使用GBDT和DNN进行推荐


In [1]:
import sys,os
import numpy as np
import pandas as pd
import math
import re

In [2]:
# 导入scikit-learn用于one-hot编码
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# movies.dat处理
    one-hot编码
    最后展示movieId和对应编码后特征array(99,)特征矩阵

### movies属性one-hot编码

In [3]:
movies_df = pd.read_table('/Users/yihaoli/Desktop/dataset/movielens-1m-dataset/movies.dat',header=None,sep='::')

/Users/yihaoli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


### 考虑年份可以作为一个特征, 所以将名称属性中的年份提取出来

In [4]:
movies_df[3] = movies_df[1].apply(lambda x : re.findall('(?<=\()[0-9]{4}(?=\))',x)[-1], 1)
movies_df[1] = movies_df[1].apply(lambda x : re.findall('^.+(?=\([0-9]{4}\))',x)[-1], 1)

In [5]:
movies_df

,0,1,2,3
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
3878,3948,Meet the Parents,Comedy,2000
3879,3949,Requiem for a Dream,Drama,2000
3880,3950,Tigerland,Drama,2000
3881,3951,Two Family House,Drama,2000


In [6]:
# 年份编码
# 需要按照顺序
# 所以操作相对多一些
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(movies_df.sort_values(3)[3].unique())
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
year_encoded = {}
years = movies_df.sort_values(3)[3].unique()
for i,y in enumerate(years):
    year_encoded[y] = onehot_encoded[i,:]
movies_df[3] = movies_df[3].map(year_encoded)

In [7]:
# 电影类目multi-hot编码
# 一个电影可能存在多个分类

# 得到电影多个标签的one-hot编码
def get_encoded(x, cat_encoded):
    temp_list = x.split('|')
    temp = np.zeros((len(onehot_encoded[0,:]),),dtype = np.float64)
    for t in temp_list:
        temp = temp + cat_encoded[t]
    return temp

cat_list = []
for m in list(movies_df[2]):
    temp_list = m.split('|')
    for t in temp_list:
        if t not in cat_list:
            cat_list.append(t)
cat_list = list(set(cat_list))
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(cat_list)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
cat_encoded = {}
for i,c in enumerate(cat_list):
    cat_encoded[c] = onehot_encoded[i,:]

movies_df[2] = movies_df[2].map(lambda x:get_encoded(x,cat_encoded))

In [8]:
# 拼接列2和列3编码得到最后特征矩阵
movies_df[4] = movies_df.apply(lambda x: np.concatenate([x[2],x[3]]),axis = 1)
movies_df = movies_df[[0,4]]
movies_df = movies_df.set_index(0)

In [9]:
movies_df

,4
0,
1,"[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ..."
5,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...
3948,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3949,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3950,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."


# users.dat处理
    one-hot编码
    最后展示userId和对应编码后特征array(,)特征矩阵

### users属性one-hot编码

In [10]:
users_df = pd.read_table('/Users/yihaoli/Desktop/dataset/movielens-1m-dataset/users.dat',header=None,sep='::')

/Users/yihaoli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [11]:
users_df

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [12]:
# 性别onehot编码
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(users_df.sort_values(1)[1].unique())
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
gender_encoded = {}
genders = users_df.sort_values(1)[1].unique()
for i,y in enumerate(genders):
    gender_encoded[y] = onehot_encoded[i,:]
users_df[1] = users_df[1].map(gender_encoded)

In [13]:
# 年龄onehot编码
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(users_df.sort_values(2)[2].unique())
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
age_encoded = {}
ages = users_df.sort_values(2)[2].unique()
for i,y in enumerate(ages):
    age_encoded[y] = onehot_encoded[i,:]
users_df[2] = users_df[2].map(age_encoded)

In [14]:
# 职业onehot编码
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(users_df.sort_values(3)[3].unique())
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
occ_encoded = {}
occs = users_df.sort_values(3)[3].unique()
for i,y in enumerate(occs):
    occ_encoded[y] = onehot_encoded[i,:]
users_df[3] = users_df[3].map(occ_encoded)

In [15]:
# 拼接列1、列2和列3编码得到最后特征矩阵
users_df[5] = users_df.apply(lambda x: np.concatenate([x[1],x[2],x[3]]),axis = 1)
users_df = users_df[[0,5]]
users_df = users_df.set_index(0)

In [16]:
users_df

,5
0,
1,"[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
5,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...
6036,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
6037,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
6038,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."


# 结合rating
### 通过movieId和userId找到对应矩阵, 将rating视为分类标签进行预测
### 选择rating最高的top5

In [17]:
rating_df = pd.read_table('/Users/yihaoli/Desktop/dataset/movielens-1m-dataset/ratings.dat',header=None,sep='::')

/Users/yihaoli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [18]:
rating_df[0] = rating_df[0].map(lambda x:users_df.loc[x,5])
rating_df[1] = rating_df[1].map(lambda x:movies_df.loc[x,4])

In [19]:
rating_df = rating_df[[0,1,2]]
rating_df[3] = rating_df.apply(lambda x: np.concatenate([x[0],x[1]]),axis = 1)

In [20]:
rating_df[3]

0          [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1          [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2          [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3          [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4          [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                                 ...                        
1000204    [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
1000205    [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
1000206    [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
1000207    [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
1000208    [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
Name: 3, Length: 1000209, dtype: object

# 特征组合与分类标签

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split

In [22]:
rating_df[3] = rating_df[3].map(lambda x: np.reshape(x,(1,129)))

In [23]:
X = np.concatenate(list(rating_df[3]),axis = 0)

In [24]:
Y = np.asarray(rating_df[2])

# GDBT训练和预测

In [85]:
# 太慢了, 换决策树先跑
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)
clf = GradientBoostingClassifier(random_state=0,tol = 1e-3)
clf.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
clf.score(X_test, y_test)

In [ ]:
clf.predict(X_test[:20])

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)
clf = tree.DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [26]:
clf.score(X_test, y_test)

0.3447149204368674

In [27]:
clf.predict(X_test[:20])

array([3, 3, 4, 4, 3, 4, 5, 4, 4, 3, 4, 5, 2, 4, 4, 3, 4, 4, 4, 2])

In [29]:
y_test[:20]

array([4, 1, 4, 5, 1, 5, 3, 4, 5, 2, 4, 5, 1, 4, 3, 5, 3, 3, 3, 2])

## 得到top5

# DNN训练和预测

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.autograd import Variable
import torch.utils.data as Data

In [31]:
#training setting
batch_size = 64

# grab train and test data
train_dataset = Data.TensorDataset(torch.tensor(torch.from_numpy(X_train),dtype=torch.float32), torch.tensor(torch.from_numpy(y_train-1),dtype=torch.long))
test_dataset = Data.TensorDataset(torch.tensor(torch.from_numpy(X_test),dtype=torch.float32), torch.tensor(torch.from_numpy(y_test-1),dtype=torch.long))

/Users/yihaoli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
/Users/yihaoli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [32]:
# Data Loader (Input Pipeline)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

In [43]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.l1 = nn.Linear(129, 256)
        self.l2 = nn.Linear(256, 64)
        self.l3 = nn.Linear(64, 5)

    def forward(self, x):
        x = x.view(-1,129) # Flattern the (n,1,28,28) to (n,784)
        x = F.relu6(self.l1(x))
        x = F.relu6(self.l2(x))
        x = F.softmax(self.l3(x))

        return x
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr= 0.01 , momentum= 0.9)

In [44]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # print(len(train_loader))
        data,target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        if batch_idx % 5000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [45]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data,volatile=True),Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data.item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [46]:
# 训练次数不够
for epoch in range(1, 50):
    train(epoch)
    test()

/Users/yihaoli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Train Epoch: 1 [0/750156 (0%)]	Loss: 1.610901
Train Epoch: 1 [320000/750156 (43%)]	Loss: 1.488907
Train Epoch: 1 [640000/750156 (85%)]	Loss: 1.564952


/Users/yihaoli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Test set: Average loss: 0.0236, Accuracy: 89226/250053 (36%)

Train Epoch: 2 [0/750156 (0%)]	Loss: 1.524418
Train Epoch: 2 [320000/750156 (43%)]	Loss: 1.512048
Train Epoch: 2 [640000/750156 (85%)]	Loss: 1.501196

Test set: Average loss: 0.0236, Accuracy: 90620/250053 (36%)

Train Epoch: 3 [0/750156 (0%)]	Loss: 1.500295
Train Epoch: 3 [320000/750156 (43%)]	Loss: 1.511681
Train Epoch: 3 [640000/750156 (85%)]	Loss: 1.450765

Test set: Average loss: 0.0235, Accuracy: 92395/250053 (37%)

Train Epoch: 4 [0/750156 (0%)]	Loss: 1.575653
Train Epoch: 4 [320000/750156 (43%)]	Loss: 1.493427
Train Epoch: 4 [640000/750156 (85%)]	Loss: 1.536489

Test set: Average loss: 0.0234, Accuracy: 93715/250053 (37%)

Train Epoch: 5 [0/750156 (0%)]	Loss: 1.515961
Train Epoch: 5 [320000/750156 (43%)]	Loss: 1.515100
Train Epoch: 5 [640000/750156 (85%)]	Loss: 1.478083

Test set: Average loss: 0.0234, Accuracy: 93963/250053 (38%)

Train Epoch: 6 [0/750156 (0%)]	Loss: 1.459001
Train Epoch: 6 [320000/750156 (43%)]	Lo

Train Epoch: 40 [320000/750156 (43%)]	Loss: 1.526196
Train Epoch: 40 [640000/750156 (85%)]	Loss: 1.549541

Test set: Average loss: 0.0233, Accuracy: 95867/250053 (38%)

Train Epoch: 41 [0/750156 (0%)]	Loss: 1.499639
Train Epoch: 41 [320000/750156 (43%)]	Loss: 1.478392
Train Epoch: 41 [640000/750156 (85%)]	Loss: 1.531389

Test set: Average loss: 0.0233, Accuracy: 96263/250053 (38%)

Train Epoch: 42 [0/750156 (0%)]	Loss: 1.489871
Train Epoch: 42 [320000/750156 (43%)]	Loss: 1.423137
Train Epoch: 42 [640000/750156 (85%)]	Loss: 1.498155

Test set: Average loss: 0.0233, Accuracy: 96196/250053 (38%)

Train Epoch: 43 [0/750156 (0%)]	Loss: 1.489006
Train Epoch: 43 [320000/750156 (43%)]	Loss: 1.529341
Train Epoch: 43 [640000/750156 (85%)]	Loss: 1.543616

Test set: Average loss: 0.0233, Accuracy: 95830/250053 (38%)

Train Epoch: 44 [0/750156 (0%)]	Loss: 1.454494
Train Epoch: 44 [320000/750156 (43%)]	Loss: 1.409032
Train Epoch: 44 [640000/750156 (85%)]	Loss: 1.493120

Test set: Average loss: 0.023

KeyboardInterrupt: 

In [52]:
i = 0
for data, target in test_loader:
    i+=1
    output = model(data)
    # print(output)
    print((output+1).data.max(1, keepdim=True)[1])
    print(target+1)
    if i ==1:
        break

tensor([[2],
        [2],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [4],
        [3],
        [3],
        [3],
        [2],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [2],
        [2],
        [4],
        [3],
        [3],
        [2],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [2],
        [3],
        [3],
        [3],
        [2],
        [3],
        [2],
        [3],
        [3],
        [2],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [2],
        [2],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [2]])
tensor([4, 1, 4, 5, 1, 5, 3, 4, 5, 2, 4, 5, 1, 4, 3, 5, 3, 3, 3, 2, 5, 1, 4, 2,
        2, 3, 5, 3, 4, 3, 4, 4, 4, 4, 5, 4, 4, 2, 5, 3, 5, 4, 1, 3, 2, 1, 3, 5,
       

/Users/yihaoli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


In [48]:
res = []
for i in range(len(output)):
    for j in range(len(output[i])):
        if output[i][j] == output[i].max():
            res.append(j+1)

In [49]:
(target+1).numpy().tolist()

[4,
 1,
 4,
 5,
 1,
 5,
 3,
 4,
 5,
 2,
 4,
 5,
 1,
 4,
 3,
 5,
 3,
 3,
 3,
 2,
 5,
 1,
 4,
 2,
 2,
 3,
 5,
 3,
 4,
 3,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 2,
 5,
 3,
 5,
 4,
 1,
 3,
 2,
 1,
 3,
 5,
 5,
 3,
 3,
 5,
 3,
 2,
 1,
 4,
 4,
 4,
 4,
 4,
 2,
 5,
 3,
 1]

In [50]:
res

[3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 5,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 3,
 4,
 3,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3]

In [60]:
output.data.max(1, keepdim=True)[1].numpy().tolist()

2

## 得到top5

# 思考
### 不使用onehot编码改用数值替换类目的效果
###  后续考虑结合协同过滤进行推荐